# A deep dive into DataFrames.jl indexing
# Part 1: indexing in DataFrames.jl by example
### Bogumił Kamiński

What are we going to cover:
* `getindex`, a.k.a. `x[...]`
* `setindex!`, a.k.a. `x[...] =`
* `broadcast`, a.k.a. `fun.(x)`
* `broadcat!`, a.k.a. `x .= ...`

Indexable types that DataFrames.jl defines:
* `DataFrame`
* `SubDataFrame`
* `DataFrameRow`
* `DataFrameRows`
* `DataFrameColumns`
* `GroupedDataFrame`
* `GroupKeys`
* `GroupKey`
* `StackedVector`
* `RepeatedVector`

### Environment setup

In [1]:
using DataFrames

In [2]:
using CSV

In [3]:
using BenchmarkTools

In [4]:
using Dates

In [5]:
using Statistics

In [6]:
ENV["COLUMNS"] = 500 # allow output up to 500 characters wide not to be truncated when displayed

500

In [7]:
ENV["LINES"] = 15 # we do not need to see too many lines in the examples we work with

15

In [8]:
df = CSV.File("fh_5yrs.csv") |> DataFrame

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


#### Warm up exercises

*Get short description of columns in our data frame*

(see https://github.com/JuliaData/DataFrames.jl/issues/2269 for a discussion of the design decisions here, feel free to comment there if you have an opinion)

*Get information about exact types of the columns stored in the data frame*

*Get names of columns as strings*

*Get names of columns as `Symbol`s*

## `getindex`

Get a single column as a whole without copying

In [9]:
unique([df.date,
        df."date",
        df[!, 1],
        df[!, :date],
        df[!, "date"]])

1-element Array{Array{Date,1},1}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

In [10]:
unique([getproperty(df, :date),
        getproperty(df, "date"),
        getindex(df, !, 1),
        getindex(df, !, :date),
        getindex(df,!, "date")])

1-element Array{Array{Date,1},1}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

Get a single column as a whole with copying

In [11]:
unique([copy(df.date),
        copy(df."date"),
        df[:, 1],
        df[:, :date],
        df[:, "date"]])

1-element Array{Array{Date,1},1}:
 [Date("2020-07-02"), Date("2020-07-01"), Date("2020-06-30"), Date("2020-06-29"), Date("2020-06-26"), Date("2020-06-25"), Date("2020-06-24"), Date("2020-06-23"), Date("2020-06-22"), Date("2020-06-19")  …  Date("2015-01-21"), Date("2015-01-20"), Date("2015-01-16"), Date("2015-01-14"), Date("2015-01-13"), Date("2015-01-12"), Date("2015-01-09"), Date("2015-01-07"), Date("2015-01-05"), Date("2015-01-02")]

Let us compare the performance of various ways to get a column without copying

In [12]:
@btime $df.date
@btime $df."date"
@btime $df[!, 1]
@btime $df[!, :date]
@btime $df[!, "date"];

  13.326 ns (0 allocations: 0 bytes)
  41.069 ns (0 allocations: 0 bytes)
  4.599 ns (0 allocations: 0 bytes)
  13.412 ns (0 allocations: 0 bytes)
  36.894 ns (0 allocations: 0 bytes)


#### Exercise

Check the same but with copying

Do you think it really matters in practice how fast is an access to column of a data frame?

Let us check how lookup speed scales with the number of columns:

In [13]:
@time df_tmp = DataFrame(ones(1, 100_000))

  0.076042 seconds (599.57 k allocations: 47.574 MiB)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
@btime $df_tmp.x100000
@btime $df_tmp."x100000"
@btime $df_tmp[!, 100000];

  15.730 ns (0 allocations: 0 bytes)
  49.898 ns (0 allocations: 0 bytes)
  5.099 ns (0 allocations: 0 bytes)


Get a single column, but take a subset of rows: you can either make a copy or get a view

In [15]:
df[1:2, :date]

2-element Array{Date,1}:
 2020-07-02
 2020-07-01

In [16]:
view(df, 1:2, :date)

2-element view(::Array{Date,1}, 1:2) with eltype Date:
 2020-07-02
 2020-07-01

this is the same as:

In [17]:
df.date[1:2]

2-element Array{Date,1}:
 2020-07-02
 2020-07-01

In [18]:
view(df.date, 1:2)

2-element view(::Array{Date,1}, 1:2) with eltype Date:
 2020-07-02
 2020-07-01

you can use `Not` for inverted selection

In [19]:
df[Not(3:end), :date]

2-element Array{Date,1}:
 2020-07-02
 2020-07-01

Get a single cell in a data frame: you can either get a value or a view

In [20]:
df[1, :date]

2020-07-02

In [21]:
view(df, 1, :date)

0-dimensional view(::Array{Date,1}, 1) with eltype Date:
Date("2020-07-02")

#### Exercise

In what case you might want to use a view instead of getting a value?

Check what is the consequence of running the following lines:

In [22]:
tmp_cell = view(df, 1, :date)

0-dimensional view(::Array{Date,1}, 1) with eltype Date:
Date("2020-07-02")

In [23]:
tmp_cell2 = getindex(df, 1, :date)

2020-07-02

In [24]:
tmp_cell[] = Date("2222-07-02")

2222-07-02

Revert the change we have just made

To conclude note that with `view` there is not difference between `!` and `:`:

In [25]:
@view df[!, 1]

6852038-element view(::Array{Date,1}, :) with eltype Date:
 2222-07-02
 2020-07-01
 2020-06-30
 2020-06-29
 2020-06-26
 ⋮
 2015-01-12
 2015-01-09
 2015-01-07
 2015-01-05
 2015-01-02

In [26]:
@view df[:, 1]

6852038-element view(::Array{Date,1}, :) with eltype Date:
 2222-07-02
 2020-07-01
 2020-06-30
 2020-06-29
 2020-06-26
 ⋮
 2015-01-12
 2015-01-09
 2015-01-07
 2015-01-05
 2015-01-02

Summary:

> passing a single column as an integer, `Symbol` or string drops one dimension of
> a data frame and allows you to select or subset a column from it

Multiple column selection options:
* a vector of `Symbol` (does not have to be a subtype of `AbstractVector{Symbol}`);
* a vector of `AbstractString` (does not have to be a subtype of `AbstractVector{<:AbstractString}`);
* a vector of `Integer` other than `Bool` (does not have to be a subtype of `AbstractVector{<:Integer}`);
* a vector of `Bool` that has to be a subtype of `AbstractVector{Bool}`;
* a regular expression, which gets expanded to a vector of matching column names;
* a `Not` expression;
* an `All` or `Between` expression;
* a colon literal :.

The type of output depends on the row selecor:
* if it is a single row you get a `DataFrameRow` (a dimension is dropped)
* if it is a collection of rows you get a data frame

Single row selection is always a view that is `DataFrameRow`:

In [27]:
df[1, [:date]]

,date
,Date
1,2222-07-02


In [28]:
@view df[1, [:date]]

,date
,Date
1,2222-07-02


Note that `DataFrameRow` is one-dimensional (as usual - single value indexing drops a dimension). You can think of it as a mutable `NamedTuple`.

Multiple row selection is a `DataFrame` for `getindex`:

In [29]:
df[1:2, 1:2]

,date,volume
,Date,Int64
1,2222-07-02,257500
2,2020-07-01,468100


In [30]:
df2 = df[!, 1:2] # this does not copy columns

,date,volume
,Date,Int64
1,2222-07-02,257500
2,2020-07-01,468100
3,2020-06-30,319100
4,2020-06-29,405500
5,2020-06-26,335100
6,2020-06-25,246800
7,2020-06-24,329200
8,2020-06-23,351800
9,2020-06-22,308300


In [31]:
df2.date === df.date

true

Using `view` creates a `SubDataFrame`

In [32]:
df3 = view(df, 1:2, 1:2)

,date,volume
,Date,Int64
1,2222-07-02,257500
2,2020-07-01,468100


In [33]:
typeof(df3)

SubDataFrame{DataFrame,DataFrames.SubIndex{DataFrames.Index,UnitRange{Int64},UnitRange{Int64}},UnitRange{Int64}}

For `view` using `:` and `!` gives you the same result:

In [34]:
dump(view(df, !, :))

SubDataFrame{DataFrame,DataFrames.Index,Base.OneTo{Int64}}
  parent: DataFrame
    columns: Array{AbstractArray{T,1} where T}((8,))
      1: Array{Date}((6852038,))
        1: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 811386
        2: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737607
        3: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737606
        4: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737605
        5: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737602
        ...
        6852034: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 735610
        6852035: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 735607
      

In [35]:
dump(view(df, :, :))

SubDataFrame{DataFrame,DataFrames.Index,Base.OneTo{Int64}}
  parent: DataFrame
    columns: Array{AbstractArray{T,1} where T}((8,))
      1: Array{Date}((6852038,))
        1: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 811386
        2: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737607
        3: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737606
        4: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737605
        5: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 737602
        ...
        6852034: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 735610
        6852035: Date
          instant: Dates.UTInstant{Day}
            periods: Day
              value: Int64 735607
      

Normally when you modify the parent of a `SubDataFrame` (or `DataFrameRow`) you may get an error when trying to access it:

In [36]:
df4 = DataFrame(reshape(1:12, 3, 4))

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [37]:
df4_v = view(df4, 1:2, 1:3)

,x1,x2,x3
,Int64,Int64,Int64
1,1,4,7
2,2,5,8


In [38]:
select!(df4, 1)

,x1
,Int64
1,1
2,2
3,3


In [39]:
df4_v

BoundsError: BoundsError: attempt to access 1-element Array{Symbol,1} at index [1:3]

A special case is when you use `:` as a column selection with a `view`. In this case the `SubDataFrame` and `DataFrameRow` always get updated with the changed columns:

In [40]:
df4 = DataFrame(reshape(1:12, 3, 4))

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11
3,3,6,9,12


In [41]:
df4_v = view(df4, 1:2, :)

,x1,x2,x3,x4
,Int64,Int64,Int64,Int64
1,1,4,7,10
2,2,5,8,11


In [42]:
select!(df4, 1, :x2 => :newcol)

,x1,newcol
,Int64,Int64
1,1,4
2,2,5
3,3,6


In [43]:
df4_v

,x1,newcol
,Int64,Int64
1,1,4
2,2,5


The reason for this behavior is that subsetting of a data frame by only rows (and taking all columns) is very common, and in this case we can create and index such views much faster. In particular `DataFrameRow`s produced by `eachrow` are efficient this way:

In [44]:
@btime mean(x -> x.open, eachrow(df))

  982.399 ms (34259693 allocations: 522.76 MiB)


298.08612911254846

In [45]:
@btime mean(i -> df[i, :open], 1:nrow(df))

  1.242 s (41111214 allocations: 627.31 MiB)


298.08612911254846

Of course, type-stable operation would be faster (but sometimes processing data row-wise is more convenient):

In [46]:
@btime mean(df.open)

  2.981 ms (1 allocation: 16 bytes)


298.08612911254846

or, if your table is not very wide (so that you are not penalized by the compilation cost of `NamedTuple`) you can use:

In [47]:
@btime mean(x -> x.open, Tables.namedtupleiterator(df))

  10.505 ms (18 allocations: 1.31 KiB)


298.0861291125547

Note though that `DataFrameRow` allows you to modify the source data frame, while iterating `NamedTuple`s is read-only (more on `setindex!` later).

In [48]:
df5 = copy(df)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


#### Exercise

In `df5` find rows in which `:high` is less than `:low` and swap them.

I give you the following column selectors. Can you tell the effect of each of them when trying to run `df[1:2, selector]`?
Write the code that tests it.

In [49]:
selectors = [Between(1, 10), Between(:low, :high), [:low, :low], All(:low, :low), All(:low, :), All()]

6-element Array{Any,1}:
 Between{Int64,Int64}(1, 10)
 Between{Symbol,Symbol}(:low, :high)
 [:low, :low]
 All{Tuple{Symbol,Symbol}}((:low, :low))
 All{Tuple{Symbol,Colon}}((:low, Colon()))
 All{Tuple{}}(())

### Indexing `GroupedDataFrame`

A `GroupedDataFrame` is a view into a data frame which defines a key allowing a fast lookup (and in particular this key is then automatically used in split-apply-combine operations with `select`, `select!`, `transform`, `transform!` and `combine`).

In [50]:
gdf = groupby(df, :symbol)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [51]:
gdf_keys = keys(gdf)

6335-element DataFrames.GroupKeys{GroupedDataFrame{DataFrame}}:
 GroupKey: (symbol = "AAAU",)
 GroupKey: (symbol = "AACG",)
 GroupKey: (symbol = "AADR",)
 GroupKey: (symbol = "AAL",)
 GroupKey: (symbol = "AAMC",)
 ⋮
 GroupKey: (symbol = "ZUO",)
 GroupKey: (symbol = "ZVO",)
 GroupKey: (symbol = "ZYME",)
 GroupKey: (symbol = "ZYNE",)
 GroupKey: (symbol = "ZYXI",)

As usual - indexing by a single value drops a dimension (you get a `SubDataFrame`)

In [52]:
gdf[1]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [53]:
gdf_keys[1]

GroupKey: (symbol = "AAAU",)

In [54]:
gdf[gdf_keys[1]]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [55]:
gdf[(symbol="AAAU",)]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [56]:
gdf[("AAAU",)]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


And indexing by a collection produces a subsetted `GroupedDataFrame`:

In [57]:
gdf[1:2]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [58]:
gdf[tuple.(["AAAU", "AACG"])]

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2222-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


## setindex!

`setindex!` is defined only for `DataFrame`, `SubDataFrame` and `DataFrameRow` (other types are read-only).

General rules:
* the dimensions of left hand side and right hand side must match;
* if right hand side has names they must match the names in left hand side.

special cases (that might be surprising):

In [9]:
x = rand(5, 5)

5×5 Array{Float64,2}:
 0.514032   0.756018   0.509723   0.239683   0.791479
 0.0816587  0.0347316  0.0556727  0.0882911  0.393864
 0.0469209  0.0887824  0.534098   0.561604   0.651513
 0.251061   0.79126    0.759959   0.920141   0.971915
 0.333516   0.605214   0.664496   0.158667   0.72381

In [10]:
df = DataFrame(x)

,x1,x2,x3,x4,x5
,Float64,Float64,Float64,Float64,Float64
1,0.514032,0.756018,0.509723,0.239683,0.791479
2,0.0816587,0.0347316,0.0556727,0.0882911,0.393864
3,0.0469209,0.0887824,0.534098,0.561604,0.651513
4,0.251061,0.79126,0.759959,0.920141,0.971915
5,0.333516,0.605214,0.664496,0.158667,0.72381


In [11]:
x[1, 1:4] = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [12]:
x

5×5 Array{Float64,2}:
 1.0        3.0        2.0        4.0        0.791479
 0.0816587  0.0347316  0.0556727  0.0882911  0.393864
 0.0469209  0.0887824  0.534098   0.561604   0.651513
 0.251061   0.79126    0.759959   0.920141   0.971915
 0.333516   0.605214   0.664496   0.158667   0.72381

In [13]:
x[1:4, 1] = [1 2; 3 4]

DimensionMismatch: DimensionMismatch("tried to assign 2×2 array to 4×1 destination")

#### Exercise

Check what happens if you try the same operation on `df`.

As a general rule - we want DataFrames.jl to match 100% what Base does. If you find cases when it does not please report an issue.

A special case, that cannot work as in Base (as `Matrix` is not resizable) is creation of new columns.

In [24]:
df = DataFrame(rand(2, 2))

,x1,x2
,Float64,Float64
1,0.415772,0.68895
2,0.374918,0.867334


In [25]:
x3 = [1, 2]

2-element Array{Int64,1}:
 1
 2

In [26]:
df.x3_1 = x3

2-element Array{Int64,1}:
 1
 2

In [27]:
df[:, :x3_2] = x3

2-element Array{Int64,1}:
 1
 2

In [28]:
df[!, :x3_3] = x3

2-element Array{Int64,1}:
 1
 2

In [29]:
df

,x1,x2,x3_1,x3_2,x3_3
,Float64,Float64,Int64,Int64,Int64
1,0.415772,0.68895,1,1,1
2,0.374918,0.867334,2,2,2


#### Exercise
Check in which cases `x3` got copied.

Let us try `setindex!` for multi-column indexing.

In [36]:
df2 = DataFrame(x1=[10,20], x3=["3","4"])

,x1,x3
,Int64,String
1,10,3
2,20,4


In [37]:
df[:, 1:2] = df2

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

#### Exercise

Check what is the state of `df` after this operation.

Now check what would happen if we switched `:` to `!` in the above operation:

Summary:
* when `:` is used to select rows it operates in-place (it works just as any row selector); except when adding a new column, in which case it copies a column;
* when `!` is used to select rows it always allocates a new column; if a single column is selected - this is a no-copy operation; if multiple columns are selected a copy is always made

## broadcast

Data frame behaves in broadcasting just like a matrix, with two exceptions:
* it forces the style of the result to be a `DataFrame`
* in particular this means that if several data frames take part in broadcasting they must have matching column names

In [34]:
df = CSV.File("fh_5yrs.csv") |> DataFrame

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-07-02,257500,17.64,17.74,17.62,17.71,17.71,AAAU
2,2020-07-01,468100,17.73,17.73,17.54,17.68,17.68,AAAU
3,2020-06-30,319100,17.65,17.8,17.61,17.78,17.78,AAAU
4,2020-06-29,405500,17.67,17.69,17.63,17.68,17.68,AAAU
5,2020-06-26,335100,17.49,17.67,17.42,17.67,17.67,AAAU
6,2020-06-25,246800,17.6,17.6,17.52,17.59,17.59,AAAU
7,2020-06-24,329200,17.61,17.71,17.56,17.61,17.61,AAAU
8,2020-06-23,351800,17.55,17.66,17.55,17.66,17.66,AAAU
9,2020-06-22,308300,17.5,17.57,17.44,17.5,17.5,AAAU


In [28]:
log.(df[!, Between(:open, :close)])

,open,high,low,close
,Float64,Float64,Float64,Float64
1,2.87017,2.87582,2.86903,2.87413
2,2.87526,2.87526,2.86448,2.87243
3,2.87074,2.8792,2.86847,2.87807
4,2.87187,2.873,2.8696,2.87243
5,2.86163,2.87187,2.85762,2.87187
6,2.8679,2.8679,2.86334,2.86733
7,2.86847,2.87413,2.86562,2.86847
8,2.86505,2.8713,2.86505,2.8713
9,2.8622,2.86619,2.85877,2.8622


In [29]:
log.(df[!, Between(:open, :adjclose)])

DomainError: DomainError with -0.8700000643730164:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).

In [30]:
filter(:adjclose => <=(0), df)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2020-05-13,507400,3.4,3.4,2.99,3.02,-0.87,HCHC
2,2020-05-12,658600,3.22,3.48,3.11,3.19,-0.918974,HCHC
3,2020-05-11,564700,3.17,3.43,3.12,3.18,-0.916093,HCHC
4,2020-05-08,264600,2.92,3.08,2.8,3.06,-0.881523,HCHC
5,2020-05-07,246900,3.06,3.06,2.67,2.8,-0.806623,HCHC
6,2020-05-06,219500,2.87,3.08,2.78,2.98,-0.858477,HCHC
7,2020-05-05,190900,2.7,2.84,2.7,2.8,-0.806623,HCHC
8,2020-05-04,139800,2.58,2.8,2.58,2.69,-0.774934,HCHC
9,2020-05-01,288800,2.64,2.77,2.54,2.65,-0.763411,HCHC


In [31]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,date,,2015-01-02,,2020-07-02,1385,,Date
2,volume,1.01541e6,1,120600.0,2156725200,,,Int64
3,open,298.086,0.001,24.95,6.91553e7,,,Float64
4,high,305.876,0.0,25.11,7.05886e7,,,Float64
5,low,291.014,0.0,24.75,6.84387e7,,,Float64
6,close,296.783,0.001,24.94,6.95136e7,,,Float64
7,adjclose,293.231,-3.77096,23.3258,6.90222e7,,,Float64
8,symbol,,AAAU,,ZYXI,6335,,String


#### Exercise

Replace non-positive values in `:adjclose` by a value in `:close` and store it in a variable `:adjclose_fix`.

and something more advanced now for each stock calculate a column with log-returns of `:close` and store it in `:close_returns`:

In [32]:
using ShiftedArrays

In [35]:
sort!(df, :date)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,AADR
2,2015-01-02,10748600,54.28,54.6,53.07,53.91,51.0799,AAL
3,2015-01-02,11500,308.0,348.59,308.0,327.18,327.18,AAMC
4,2015-01-02,11400,3.99,4.03,3.98,4.03,3.91772,AAME
5,2015-01-02,898900,30.81,30.86,30.04,30.62,30.0588,AAN
6,2015-01-02,184600,11.28,11.28,10.72,10.79,10.79,AAOI
7,2015-01-02,90700,22.55,22.68,21.6,21.93,20.9958,AAON
8,2015-01-02,509800,160.85,162.5,157.47,158.56,156.251,AAP
9,2015-01-02,53204600,111.39,111.44,107.35,109.33,99.9459,AAPL


In [36]:
gdf = groupby(df, :symbol)

,date,volume,open,high,low,close,adjclose,symbol
,Date,Int64,Float64,Float64,Float64,Float64,Float64,String
1,2018-08-15,27300,11.84,11.84,11.74,11.74,11.74,AAAU
2,2018-08-16,428400,11.78,11.8,11.74,11.74,11.74,AAAU
3,2018-08-17,52400,11.8,11.82,11.77,11.82,11.82,AAAU
4,2018-08-20,28700,11.88,11.91,11.85,11.9,11.9,AAAU
5,2018-08-21,30600,11.92,11.95,11.89,11.93,11.93,AAAU
6,2018-08-22,101200,11.98,11.98,11.94,11.96,11.96,AAAU
7,2018-08-23,54800,11.91,11.92,11.85,11.85,11.85,AAAU
8,2018-08-24,106500,11.97,12.08,11.96,12.05,12.05,AAAU
9,2018-08-27,295100,12.06,12.13,12.06,12.1,12.1,AAAU


In [37]:
transform!(gdf, :close => (x -> log.(x ./ lag(x))) => :close_returns)

,symbol,date,volume,open,high,low,close,adjclose,close_returns
,String,Date,Int64,Float64,Float64,Float64,Float64,Float64,Float64?
1,AADR,2015-01-02,2000,37.25,37.25,36.64,36.64,35.3998,missing
2,AAL,2015-01-02,10748600,54.28,54.6,53.07,53.91,51.0799,missing
3,AAMC,2015-01-02,11500,308.0,348.59,308.0,327.18,327.18,missing
4,AAME,2015-01-02,11400,3.99,4.03,3.98,4.03,3.91772,missing
5,AAN,2015-01-02,898900,30.81,30.86,30.04,30.62,30.0588,missing
6,AAOI,2015-01-02,184600,11.28,11.28,10.72,10.79,10.79,missing
7,AAON,2015-01-02,90700,22.55,22.68,21.6,21.93,20.9958,missing
8,AAP,2015-01-02,509800,160.85,162.5,157.47,158.56,156.251,missing
9,AAPL,2015-01-02,53204600,111.39,111.44,107.35,109.33,99.9459,missing


note that `GroupedDataFrame` got automatically updated

In [38]:
gdf

,symbol,date,volume,open,high,low,close,adjclose,close_returns
,String,Date,Int64,Float64,Float64,Float64,Float64,Float64,Float64?
1,AAAU,2018-08-15,27300,11.84,11.84,11.74,11.74,11.74,missing
2,AAAU,2018-08-16,428400,11.78,11.8,11.74,11.74,11.74,0.0
3,AAAU,2018-08-17,52400,11.8,11.82,11.77,11.82,11.82,0.00679119
4,AAAU,2018-08-20,28700,11.88,11.91,11.85,11.9,11.9,0.00674538
5,AAAU,2018-08-21,30600,11.92,11.95,11.89,11.93,11.93,0.00251789
6,AAAU,2018-08-22,101200,11.98,11.98,11.94,11.96,11.96,0.00251149
7,AAAU,2018-08-23,54800,11.91,11.92,11.85,11.85,11.85,-0.00923985
8,AAAU,2018-08-24,106500,11.97,12.08,11.96,12.05,12.05,0.0167368
9,AAAU,2018-08-27,295100,12.06,12.13,12.06,12.1,12.1,0.00414081


In [45]:
gdf2 = transform!(groupby(df, :symbol), :close => (x -> log.(x ./ lag(x))) => :close_returns, ungroup=false)

,symbol,adjclose,date,volume,open,high,low,close,close_returns
,String,Float64,Date,Int64,Float64,Float64,Float64,Float64,Float64?
1,AAAU,11.74,2018-08-15,27300,11.84,11.84,11.74,11.74,missing
2,AAAU,11.74,2018-08-16,428400,11.78,11.8,11.74,11.74,0.0
3,AAAU,11.82,2018-08-17,52400,11.8,11.82,11.77,11.82,0.00679119
4,AAAU,11.9,2018-08-20,28700,11.88,11.91,11.85,11.9,0.00674538
5,AAAU,11.93,2018-08-21,30600,11.92,11.95,11.89,11.93,0.00251789
6,AAAU,11.96,2018-08-22,101200,11.98,11.98,11.94,11.96,0.00251149
7,AAAU,11.85,2018-08-23,54800,11.91,11.92,11.85,11.85,-0.00923985
8,AAAU,12.05,2018-08-24,106500,11.97,12.08,11.96,12.05,0.0167368
9,AAAU,12.1,2018-08-27,295100,12.06,12.13,12.06,12.1,0.00414081


#### Exercise (advanced)

Check our codes to make sure that in each group first and only the fist element of `:close_returns` is `missing`

#### Exercises

Check what happens if you try to broadcast a sum of a 1-row `DataFrame` with an array having multiple rows

Check what happens if you try to broadcast a sum of a 0-row `DataFrame` with an array having multiple rows

Check what happens when you try to broadcast a sum of a `DataFrame` with a 3D array.

Check if broadcasting is defined for `DataFrameRow`.